# [Largest Rectangle in Histogram](https://leetcode.com/problems/largest-rectangle-in-histogram/description/)

- **Given**
    + Array of integers heights `H` representing a histogram, where the width of each bar is 1
- **Return** the area of the largest rectangle in the histogram.

#### Examples

```
Input: heights = [2,1,5,6,2,3]
Output: 10
```

<img src="./assets/2.png" width="500"/>

```
Input: heights = [2,4]
Output: 4
```

<img src="./assets/4.png" width="300"/>

## Solution 1 - Monotonic Stack O(N)

- Each index i will have a max area rectangle limit by 2 pillars l and r -> find the i with maxarea

<img src="./assets/5.png" width="300"/>


- Build a **Monotonic increasing stack**, max area of an index `i` (stck.top()) is limit by
    + index `l` prev elem in the stack,
    + index `r` adding elem
    + area = H[i] * (r-l-1)
    
```
Increasing stack: <l> <i>[top] <r>
```

#### Code

```Cpp
class Solution {
public:
    int largestRectangleArea(vector<int> &H) {
        // Add a dummy value (0) to pop all elem 
        H.push_back(0);

        // min <<< max [back] H[r]
        vector<int> stck;
        int ans = 0;
        for(int r=0; r<H.size(); ++r) {
            // Pop stack
            while(!stck.empty() && H[stck.back()] >= H[r]) {
                int i = stck.back();
                stck.pop_back();

                int l = stck.empty() ? -1 : stck.back();
                int area = (r-l-1) * H[i];

                // Relax ans
                ans = max(ans, area);
            }

            // Push stck
            stck.push_back(r);
        }

        return ans;
    }
};
```

## Solution 2 - Divide and Conquer 

#### Simple (TLE)

```Cpp
class Solution {
private:
    int max3(int a, int b, int c) {
        return max(max(a,b), c);
    }

    int get(const vector<int> &H, int l, int r) {
        if(l > r) return 0;

        // Find the min height idx in [l,r]
        int p = l;
        for(int i=l; i<=r; ++i) {
            if(H[i] < H[p]) p = i;
        }

        // Max area in [l,r]
        int area = (r-l+1)*H[p]; 

        // Return
        return max3(
            area,
            get(H, l, p-1),
            get(H,p+1,r));
    }
public:
    int largestRectangleArea(const vector<int> &H) {
        return get(H,0,H.size()-1);
    }
};
```

#### O(Nlog(N))

```Cpp
class Solution {
private:
    int max3(int a, int b, int c) {
        return max(max(a,b), c);
    }

    int get(const vector<int> &H, int l, int r) {
        if(l > r) return 0;
        if(l == r) return H[l];

        // Get middle
        int m = l+(r-l)/2;

        // Expand from the middle to find the max area containing [i,j]
        int i = m, j = m+1;
        int area = 0, h = min(H[i], H[j]);
        while(i >= l && j <= r) {
            // Update h
            h = min(h, min(H[i], H[j]));
            
            // Calculate area
            area = max(area, (j-i+1) * h);

            // Expand Rules*
            if (i == l) ++j;
            else if (j == r) --i;
            else {
                // if both sides have not reached the boundary,
                // compare the outer bars and expand towards the bigger side
                if (H[i-1] > H[j+1]) --i;
                else ++j;
            }
        }

        return max3(
            area,
            get(H, l,m),
            get(H, m+1,r)
        );

    }
public:
    int largestRectangleArea(const vector<int> &H) {
        return get(H, 0, H.size()-1);
    }
};
```